# GPS Simulation Project

### Haversine Function

In [1]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d


# Sample for filling the function print( distance((lat1, long1), (lat2, long2)) )

### Reading points from digital maps

In [9]:
from pykml import parser
import pandas as pd

filename='KirunaUmeaGPSSIM.kml'
with open(filename) as f:
    folder = parser.parse(f).getroot().Document.Folder

plnm = []
cordi = []
for pm in folder.Placemark:
    plnm1=pm.name
    plcs1=pm.Point.coordinates
    plnm.append(plnm1.text)
    cordi.append(plcs1.text)
    
db=pd.DataFrame()
db['place_name']=plnm
db['cordinates']=cordi

def dump(obj):
    for attr in dir(obj):
        if hasattr( obj, attr ):
            print( "obj.%s = %s" % (attr, getattr(obj, attr)))
#the kml output plnm1 and plcs1 are objects actually which can be viewed using the dump function from [SO](https://blender.stackexchange.com/questions/1879/is-it-possible-to-dump-an-objects-properties-and-methods)

db['Longitude'], db['Latitude'],db['value'] = zip(*db['cordinates'].apply(lambda x: x.split(',', 2)))
db.drop(['value','cordinates'],axis=1,inplace=True)

db

,place_name,Longitude,Latitude
0,01. Point 1,\n 20.1990912,67.8683098
1,02. Point 2,\n 22.1653846,65.5841464
2,03. Point 3,\n 20.2935445,63.8176726


### Creating appropirate dataBase for the calculating points

In [10]:
#db1 = db[['Longitude','Latitude']]
db.Longitude.str.replace('\n','')  # Eliminating the \n from the Longitude
db['Lon_int'] = db.Longitude.astype(float)
db['Lat_int'] = db.Latitude.astype(float) 
db1 = db[['place_name','Lat_int','Lon_int']]
db1.info

<bound method DataFrame.info of     place_name    Lat_int    Lon_int
0  01. Point 1  67.868310  20.199091
1  02. Point 2  65.584146  22.165385
2  03. Point 3  63.817673  20.293544>

### Calculating the distance between any two sequenced points

In [35]:
[m,n] = db1.shape
 
Distances = []
for i in range(m-1):
    TempResult = distance((db1.iloc[i,1],db1.iloc[i,2]),(db1.iloc[i+1,1],db1.iloc[i+1,2]))
    Distances.append(["Track {}".format(i),TempResult])
    
#Converting Distances List to Pandas Dataframe
dbDistances = pd.DataFrame(Distances)

### Representing the distance on each track (km)

In [36]:
Distances

[['Track 0', 268.2449176213075], ['Track 1', 215.60125293247705]]

### Total distances

In [51]:
TotalDistance = dbDistances.sum()
TotalDistance

0    Track 0Track 1
1           483.846
dtype: object

# GPS simulation 


### Calculating angle  between two sequential points which is called BEARING
#### Function of Bearing (Horisontal Bearing)

In [13]:
def calcBearing(lat1, lon1, lat2, lon2):
    dLon = lon2 - lon1
    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) \
        - math.sin(lat1) * math.cos(lat2) * math.cos(dLon)
    return math.atan2(y, x)

In [14]:
Bearings = []
for i in range(m-1):
    TempBearing = calcBearing(db1.iloc[i,1],db1.iloc[i,2],db1.iloc[i+1,1],db1.iloc[i+1,2])
    Bearings.append(["Track {}".format(i),TempBearing])

### Representing the bear on each track

In [15]:
Bearings

[['Track 0', -1.0777396176995913], ['Track 1', -2.5022122769128305]]

# 1. Scenario and assumptions

-  Speed has been assumed as 80 km/h.
-  Required time for passing Kiruna-lulea: 5 hours
-  Required time for passing Lulea-Umea: 4 hours
-  starting point is Kiruna and lulea will be pass and finally the distination is Umea


## Assumption of Speed & Time

In [56]:
Speed = 80
TimeKirunaLulea = 5
TimeLuleaUmea = 4

## Distance calculation without puting any NOISE

In [58]:
DistanceKirunaLulea = Speed*TimeKirunaLulea
DistanceLuleaUmea = Speed*TimeLuleaUmea
TotalDistanceSim = DistanceKirunaLulea + DistanceLuleaUmea
print('Total Distance with out any Noise: ',TotalDistanceSim)

Total Distance with out any Noise:  720


It has been concluded from the Simulation and the direct calculation (720 km and 482 km respectively). So, the conclusion is not appropriate enough.

# 2. Scenario and assumptions

-  In this scenario speeds change and timing will chagne as a consequence.

In [65]:
# Speeds for different tracks

import numpy as np
SpeedsKiurnaLulea = np.array([50,80,70,90,100,70,50,40,80,100])
SpeedsLuleaUmea = np.array([50,100,80,90,100,110,80,90,100,80])        